处理MUTAG数据

In [1]:
import dgl
import torch as th
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# 全局信息
node_label = ['C', 'N', 'O', 'F', 'I', 'Cl', 'Br']
node_feature_dim = len(node_label)
edge_label = ['aromatic', 'single', 'double', 'triple']
edge_feature_dim = len(edge_label)

In [7]:
# 生成图结构
graphs = []
f = open('./data/MUTAG_graph_indicator.txt', 'r')
graph_indicator = []
for line in f:
    graph_indicator.append(int(line))
graph_num = graph_indicator[-1]
total_node_num = len(graph_indicator)
f.close()

mutag_a = open('./data/MUTAG_A.txt', 'r')
total_edge_num = 0
graphs_edges = [[[],[]] for _ in range(graph_num)]
for line in mutag_a:
    total_edge_num += 1
    line = line.strip().replace(' ', '')
    src_id = int(line.split(',')[0]) # 1-base
    dst_id = int(line.split(',')[1])
    cur_graph_id = graph_indicator[src_id - 1] - 1 # 0-base
    cur_graph_id_min = graph_indicator.index(cur_graph_id + 1) + 1 # 1-base
    src_id -= cur_graph_id_min
    dst_id -= cur_graph_id_min
    graphs_edges[cur_graph_id][0].append(src_id)
    graphs_edges[cur_graph_id][1].append(dst_id)
mutag_a.close()
for graph_id in range(graph_num):
    g = dgl.graph(tuple(graphs_edges[graph_id]))
    graphs.append(g)
    

In [8]:
# 为图添加特征
node_feature = th.zeros((total_node_num, node_feature_dim))
edge_feature = th.zeros((total_edge_num, edge_feature_dim))
with open('./data/MUTAG_node_labels.txt', 'r') as f:
    nid = 0
    for line in f:
        node_feature[nid][int(line)] = 1
        nid += 1
with open('./data/MUTAG_edge_labels.txt', 'r') as f:
    eid = 0
    for line in f:
        edge_feature[eid][int(line)] = 1
        eid += 1
cur_nid_min = 0
cur_eid_min = 0
for graph_id, g in enumerate(graphs):
    g.ndata['feature'] = node_feature[cur_nid_min:cur_nid_min + g.num_nodes()]
    cur_nid_min += g.num_nodes()
    g.edata['feature'] = edge_feature[cur_eid_min:cur_eid_min + g.num_edges()]
    cur_eid_min += g.num_edges()

In [9]:
# 获取graph label
graph_labels = th.zeros(graph_num)
with open('./data/MUTAG_graph_labels.txt', 'r') as f:
    graph_id = 0
    for line in f:
        if int(line) == 1:
            graph_labels[graph_id] = 1
        graph_id += 1

In [10]:
# 存储
th.save(graph_labels, './data/graph_labels.pth')
dgl.save_graphs('./data/graphs.dgl', graphs)

In [15]:
a = th.tensor((0))
a + 0.1

tensor(0.1000)